In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

import yfinance as yf

In [2]:
# 시각화 관련 ##
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [3]:
##절대경로 참조
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

from library.quant_functions import anal_funcs, data_funcs

In [6]:
df = data_funcs.get_df('^IXIC')

[*********************100%%**********************]  1 of 1 completed


In [7]:
df

,Open,High,Low,Close,Volume
Date,,,,,
1971-02-05,100.000000,100.000000,100.000000,100.000000,0
1971-02-08,100.839996,100.839996,100.839996,100.839996,0
1971-02-09,100.760002,100.760002,100.760002,100.760002,0
1971-02-10,100.690002,100.690002,100.690002,100.690002,0
1971-02-11,101.449997,101.449997,101.449997,101.449997,0
...,...,...,...,...,...
2023-08-31,14041.540039,14114.669922,14010.480469,14034.969727,4568610000
2023-09-01,14129.959961,14149.620117,13982.419922,14031.809570,4033960000
2023-09-05,13994.540039,14060.849609,13945.650391,14020.950195,4379790000
